In [1]:
### Import

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import pylab as P
import statistics as st
import geopandas as gpd
from geopandas import GeoDataFrame

from shapely.geometry import Point
#from math import *
import math

from datetime import date

import seaborn as sn
import statsmodels.api as sm
import statsmodels.formula.api as smf

import folium

In [2]:
map_osm = folium.Map(location=[-5, 20], zoom_start=5)
map_osm

In [3]:
### Example of data input and visualisation

data=pd.read_csv("C:\\Users\\Utilisateur\\Desktop\\TUM\\Tech Challenge\\TechChallenge_Results.csv",sep=";")
data.dropna(inplace=True)
data

,Medicine,Stock_t,Stock_t+1,Stock_t+2,Stock_t+3,Stock_t+4,Demand_t,Demand_t+1,Demand_t+2,Demand_t+3,Demand_t+4,x,y
0,Medicine 1,315,318,318,315,300,318,300,335,333,336,-3.524163,23.599510
1,Medicine 2,350,308,322,340,311,317,331,302,348,337,-3.524163,23.599510
2,Medicine 3,310,314,304,307,350,301,331,330,317,347,-3.524163,23.599510
3,Medicine 4,328,302,324,312,314,339,331,304,343,301,-3.524163,23.599510
4,Medicine 5,316,314,300,337,312,315,338,336,345,344,-3.524163,23.599510
5,Medicine 1,341,316,327,300,347,335,334,327,327,332,-4.321706,15.312597
6,Medicine 2,313,349,303,317,304,321,350,347,317,300,-4.321706,15.312597
7,Medicine 3,333,341,310,307,327,308,345,307,329,343,-4.321706,15.312597
8,Medicine 4,350,338,334,339,337,344,348,329,338,340,-4.321706,15.312597
9,Medicine 5,327,322,316,338,331,305,334,321,319,324,-4.321706,15.312597


In [4]:
data_base=data.copy()
t="t"
medicine="Medicine 1"
x,y=-3.524163,23.599510
"""
scale=data_base[str(t)].loc[(data_base["Medicine"]==str(medicine))&(data_base["x"]==float(x))]
scale
"""
Vector_x=list(data_base["x"].loc[data_base["Medicine"]==str(medicine)])
Vector_y=list(data_base["y"].loc[data_base["Medicine"]==str(medicine)])
x,y=Vector_x[0],Vector_y[0]
x,y

(-3.5241628, 23.5995098)

In [5]:
### The scale of the circles should be modified to become more suitable for vizualisation


def Prediction_stock_xy(x,y,t,medicine,data_base):
    """
    Input:
    
    data_base as input (the one above)
    medicine: the medicine we want to visualise
    t: the date
    x,y: geographical coordinates
    
    Output: the future stocks forecasted for the health center (x,y)
    (taking into account incoming goods and forecasted demand)
    
    Color: 
    _red: expected scarcity
    _green: expected surplus
    
    => The bigger the circle, the bigger the expected scarcity/surplus
    
    """
      
    data=data_base.loc[(data_base["Medicine"]==str(medicine))&(data_base["x"]==float(x))&(data_base["y"]==float(y))]
    
    stock='Stock_{}'.format(t)
    demand='Demand_{}'.format(t)
    stock_net_t=float(data[stock]-data[demand])
                                 
    legend={}
    legend["Name"]=medicine
    legend["Date"]=t
    legend["x,y"]=x,y
    legend["quantity"]=stock_net_t
    
    scale=stock_net_t
  
    if scale>=0:
        color='green'
    else:
        color='red'

    scale=abs(scale)
    map_osm = folium.Map(location=[-5, 20], zoom_start=5)
    map_osm.add_child(folium.CircleMarker(location=[x,y], popup=legend, 
                                            fill_color='#132b5e', color=color,radius=scale))
   

    return(map_osm)

def Prediction(t,medicine,data_base):
    """
    Input:
    
    data_base as input (the one above)
    medicine: the medicine we want to visualise
    t: the date
    
    Output: for the different health centers, the future stocks forecasted 
    (taking into account incoming goods and forecasted demand)
    
    Color: 
    _red: expected scarcity
    _green: expected surplus
    
    => The bigger the circle, the bigger the expected scarcity/surplus
    
    """
    
    
    map_osm = folium.Map(location=[-5, 20], zoom_start=5)
    if medicine=="all":
        Vector_x=list(data_base["x"].unique)
        Vector_y=list(data_base["y"].unique)
      
    else:
        Vector_x=list(data_base["x"].loc[data_base["Medicine"]==str(medicine)])
        Vector_y=list(data_base["y"].loc[data_base["Medicine"]==str(medicine)])
      
    for place in range(len(Vector_x)):
      
        x,y=Vector_x[place],Vector_y[place]
        
        data=data_base.loc[(data_base["Medicine"]==str(medicine))
                            &(data_base["x"]==float(x))
                         &(data_base["y"]==float(y))]
    
        stock='Stock_{}'.format(t)
        demand='Demand_{}'.format(t)
        stock_net_t=float(data[stock]-data[demand])
        legend={}
        legend["Name"]=medicine
        legend["Date"]=t
        legend["x,y"]=x,y
        legend["quantity"]=abs(stock_net_t)
        
        scale=stock_net_t

        if scale>=0:
            color='green'
        else:
            color='red'

        scale=abs(scale)

        map_osm.add_child(folium.CircleMarker(location=[x,y], popup=legend, 
                                                fill_color='#132b5e', color=color,radius=scale))
                          
        
    return(map_osm)


In [6]:
t="t+3"
medicine="Medicine 3"


#Prediction_stock_xy(x,y,t,"Medicine 1",data)
Prediction(t,medicine,data)